In [1]:
import os
import numpy as np
from matplotlib import pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve
import scipy.stats as st
import pandas as pd

In [2]:
def preprocess(array):
    rows = []
    for i in range(0, len(array)):
        if i % 2 == 1:
            continue
        complex_number = array[i] + 1j*array[i+1]
        rows.append(abs(complex_number))
    return rows

In [6]:
# samples are stored as files with 100 rows in a folder
sand_path = 'sixth-scan/sand/parsed/'
cybertruck_path = 'sixth-scan/cybertruck/parsed/'

In [24]:
# read mined data for machine learning
rows = []
for file in os.listdir(sand_path):
    with open(sand_path + file) as file:
        for row in np.loadtxt(file, comments='%', delimiter=',').transpose():
            rows.append(row)
pd.DataFrame(rows)

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,100000.000000,1.735673e+08,3.470347e+08,5.205020e+08,6.939694e+08,8.674367e+08,1.040904e+09,1.214371e+09,1.387839e+09,1.561306e+09,...,6.938794e+09,7.112261e+09,7.285729e+09,7.459196e+09,7.632663e+09,7.806131e+09,7.979598e+09,8.153065e+09,8.326533e+09,8.500000e+09
1,0.000022,-7.609917e-02,-3.031377e-03,-2.540761e-02,1.713915e-01,-1.641205e-01,-1.274923e-02,8.620533e-02,2.441553e-02,-6.312399e-02,...,1.365985e-03,-6.175515e-03,6.694518e-03,-1.341763e-02,-1.563588e-03,5.636626e-03,-6.337750e-03,1.246191e-02,-4.309559e-03,1.053871e-02
2,0.000003,-1.682318e-02,-6.639751e-04,4.556763e-02,-2.718390e-02,-1.661418e-01,1.449410e-01,1.540667e-02,-9.187876e-02,-4.603894e-02,...,-7.168491e-03,8.349272e-03,-2.432997e-03,-5.521282e-03,1.520933e-02,-5.957474e-03,8.362709e-03,1.364365e-03,-4.495531e-03,3.478972e-04
3,0.000009,-8.340810e-02,-3.335110e-03,-1.622079e-02,1.715296e-01,-1.377191e-01,-4.483456e-02,1.017815e-01,2.287667e-02,-7.994725e-02,...,-1.375647e-02,7.776907e-03,1.493465e-03,-9.945442e-03,2.923153e-03,-1.743221e-03,4.765089e-03,3.433281e-03,4.893684e-03,6.913052e-03
4,0.000029,-2.826584e-02,-1.294550e-03,3.405513e-02,-2.560093e-02,-1.523937e-01,1.522483e-01,4.727052e-02,-9.096214e-02,-1.782391e-02,...,4.311114e-04,8.087965e-03,-9.270106e-03,4.711222e-03,8.801851e-03,-1.328843e-03,7.945775e-03,-2.317625e-03,3.705010e-03,-7.845369e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10045,0.000012,-2.114816e-02,2.128803e-04,3.550586e-02,-2.460102e-02,-1.505671e-01,1.448497e-01,1.238443e-02,-8.223036e-02,-3.954278e-02,...,-4.192100e-03,4.186219e-03,-6.069081e-03,1.101750e-03,9.442802e-03,-6.641002e-03,1.368497e-02,-5.701165e-03,2.543513e-03,-8.091270e-03
10046,0.000031,-8.439793e-02,-4.254584e-03,-2.461078e-02,1.607232e-01,-1.854539e-01,-3.244949e-02,8.197679e-02,8.980233e-03,-6.599146e-02,...,3.636425e-03,1.211483e-03,-1.667068e-03,-9.687414e-04,-4.899441e-03,6.852700e-03,-3.189909e-03,7.991355e-03,6.724527e-03,1.143881e-03
10047,-0.000035,-1.730404e-02,-1.129465e-03,4.270761e-02,-9.956042e-03,-1.163296e-01,1.435856e-01,7.829221e-03,-7.437154e-02,-3.602239e-02,...,5.307745e-03,-3.964871e-03,1.399232e-03,-2.842640e-03,5.356132e-03,2.747269e-03,1.447533e-03,5.395726e-03,-5.242451e-03,-3.343028e-03
10048,0.000040,-8.626898e-02,-3.826115e-03,-2.572865e-02,1.632750e-01,-1.790176e-01,-1.972608e-02,8.138124e-02,1.169033e-02,-6.396127e-02,...,7.693954e-04,7.393336e-03,-6.360136e-03,1.790333e-03,-4.134177e-03,1.293520e-03,3.049928e-03,1.511390e-03,1.006714e-02,2.921404e-04


In [37]:
# export model
pkl_filename = "pickle_rick_model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(model, file)